In [1]:
import chess.pgn
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import random
import datetime
import chess
import chess.engine
import itertools
import random
import pandas as pd
import io
from stockfish import Stockfish
from random import choice
from copy import deepcopy

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
square_index = {
    "a": 0,
    "b": 1,
    "c": 2,
    "d": 3,
    "e": 4,
    "f": 5,
    "g": 6,
    "h": 7,
}


def square_to_index(square):
    box = chess.square_name(square)
    return (8 - int(box[1])), square_index[box[0]]


def board_to_binary(board):
    tensor_board = np.zeros((14, 8, 8), dtype=np.byte)

    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            tensor_board[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            tensor_board[piece + 5][7 - idx[0]][idx[1]] = 1

    turn = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        tensor_board[12][i][j] = 1

    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        tensor_board[13][i][j] = 1

    board.turn = turn

    return tensor_board


def move_to_matrix(move):
    to = np.zeros((8, 8), dtype=np.byte)
    from_ = np.zeros((8, 8), dtype=np.byte)
    to[7 - move.to_square // 8, move.to_square % 8] = 1
    from_[7 - move.from_square // 8, move.from_square % 8] = 1
    return from_.flatten(), to.flatten()

In [3]:
class vit:
    def __init__(self, name, elo, path):
        self.elo = elo
        self.name = name
        self.model_path = f"{path}vit_{elo}_Elo"
        self.from_ = tf.keras.models.load_model(self.model_path + "_from.tf")
        self.to = tf.keras.models.load_model(self.model_path + "_to.tf")

    def get_move_probabilities(self, board, side):
        bin_board = board_to_binary(board)

        if side == "black":
            bin_board *= -1

        bin_board = np.moveaxis(bin_board, 0, -1)

        assert bin_board.shape == (8, 8, 14)

        from_board = np.array(self.from_(np.array([bin_board]))) 
        # from_board = np.array(self.from_(np.array([bin_board]))) ** 3
        to_board = np.array(self.to(np.array([bin_board]))) 
        # to_board = np.array(self.to(np.array([bin_board]))) ** 3

        # from_board /= from_board.sum()
        # to_board /= to_board.sum()

        return from_board, to_board

    def get_move(self, board, side):
        valid_moves = list(board.legal_moves)

        if len(valid_moves) == 0:
            return None  # No valid moves

        # Check if theres a checkmate, then dont have to predict :
        bd = board.copy()
        for move in valid_moves:
            board.push(move)
            if board.is_checkmate():
                move = board.pop()
                return str(move)
            _ = board.pop()

        # Check for a queen promotion
        for move in valid_moves:
            if str(move)[-1] == "q":
                return str(move)

        from_, to = self.get_move_probabilities(board, side)
        max_avg = -np.inf
        best_move = None
        from_move = None

        for move in valid_moves:
            move_matrix_from, _ = move_to_matrix(move)
            from_avg = np.max(from_ * move_matrix_from)

            if from_avg > max_avg:
                max_avg = from_avg
                from_move = str(move)[:2]

        for move in valid_moves:
            max_to_avg = -np.inf
            if str(move)[:2] == from_move:
                _, move_matrix_to = move_to_matrix(move)
                to_avg = np.max(to * move_matrix_to)

                if to_avg > max_to_avg:
                    max_to_avg = to_avg
                    best_move = str(move)

        return best_move

In [4]:
class cnn:
    def __init__(self, name, elo, path):
        self.elo = elo
        self.name = name
        self.model_path = f"{path}CNN_{elo}_Elo"
        self.from_ = tf.keras.models.load_model(self.model_path + "_from.tf")
        self.to = tf.keras.models.load_model(self.model_path + "_to.tf")

    def get_move_probabilities(self, board, side):
        bin_board = board_to_binary(board)

        if side == "black":
            bin_board *= -1

        from_board = np.array(self.from_(np.array([bin_board]))) ** 3
        to_board = np.array(self.to(np.array([bin_board]))) ** 3

        from_board /= from_board.sum()
        to_board /= to_board.sum()

        return from_board, to_board

    def get_move(self, board, side):
        valid_moves = list(board.legal_moves)

        if len(valid_moves) == 0:
            return None  # No valid moves

        # Check if theres a checkmate, then dont have to predict :
        bd = board.copy()
        for move in valid_moves:
            board.push(move)
            if board.is_checkmate():
                move = board.pop()
                return str(move)
            _ = board.pop()

        # Check for a queen promotion
        for move in valid_moves:
            if str(move)[-1] == "q":
                return str(move)

        from_, to = self.get_move_probabilities(board, side)
        max_avg = -np.inf
        best_move = None
        from_move = None

        for move in valid_moves:
            move_matrix_from, _ = move_to_matrix(move)
            from_avg = np.max(from_ * move_matrix_from)

            if from_avg > max_avg:
                max_avg = from_avg
                from_move = str(move)[:2]

        for move in valid_moves:
            max_to_avg = -np.inf
            if str(move)[:2] == from_move:
                _, move_matrix_to = move_to_matrix(move)
                to_avg = np.max(to * move_matrix_to)

                if to_avg > max_to_avg:
                    max_to_avg = to_avg
                    best_move = str(move)

        return best_move

In [5]:
class random_agent:
    def __init__(self, name):
        self.name = name
    
    def get_move(self, board, side):
        valid_moves = list(board.legal_moves)
        if len(valid_moves) == 0:
            return None
        return str(choice(valid_moves))

In [6]:
class stockfish:
    def __init__(self, name, elo, depth):
        self.elo = elo
        self.name = name
        self.config = {"UCI_LimitStrength": "true", 
                       "UCI_Elo": elo,
                       "depth" : depth }
       
        self.stockfish = Stockfish(path="stockfish/stockfish-windows-x86-64-avx2", depth = depth, parameters=self.config)
        self.stockfish.set_elo_rating(self.elo)

    def get_move(self, board, side):
        self.stockfish.set_fen_position(board.fen())
        move = self.stockfish.get_best_move()
        if move is None:
            print("NONE")
        return str(move)


In [7]:
def setup_board(move_list):
    pgn = io.StringIO(move_list)
    game = chess.pgn.read_game(pgn)

    fen = ""

    while game.next():
        game = game.next()

    fen = game.board().fen()

    return chess.Board(fen)

def fen_to_board(fen):
    return chess.Board(fen)



def play_games(model1, model2, game_num, move_dataframe):
    assert game_num < 2700

    model1_wins_white = 0
    model1_wins_black = 0

    model2_wins_white = 0
    model2_wins_black = 0

    draw = 0

    for game_c in tqdm(range(game_num)):
        # board = setup_board(move_dataframe["moves"][game_c])
        board = fen_to_board(move_dataframe["FEN"][game_c])
        # Model  1 is white
        while not board.is_game_over():
            if board.turn == chess.WHITE:
                move = model1.get_move(board, "white")
            else:
                move = model2.get_move(board, "black")
            board.push_san(move)

        if board.result() == "1-0":
            model1_wins_white += 1
        elif board.result() == "0-1":
            model2_wins_black += 1
        else:
            draw += 1

    for game_c in tqdm(range(game_num)):
        board = fen_to_board(move_dataframe["FEN"][game_c])
        # Model  2 is white
        while not board.is_game_over():
            if board.turn == chess.WHITE:
                move = model2.get_move(board, "white")
            else:
                move = model1.get_move(board, "black")
            
            board.push_san(move)

        if board.result() == "1-0":
            model2_wins_white += 1
        elif board.result() == "0-1":
            model1_wins_black += 1
        else:
            draw += 1

    total_games = game_num * 2

    return {
        "model1_name" : model1.name,
        "model2_name" : model2.name,
        "model1_wins_white": model1_wins_white,
        "model1_wins_black": model1_wins_black,
        "model2_wins_white": model2_wins_white,
        "model2_wins_black": model2_wins_black,
        "draw": draw,
        "total_games": total_games,
        "game_num": game_num
    }

def get_match_details(match_dict):
    model1 = match_dict["model1_name"]
    model2 = match_dict["model2_name"]
    model1_wins_white = match_dict["model1_wins_white"]
    model1_wins_black = match_dict["model1_wins_black"]
    model2_wins_white = match_dict["model2_wins_white"]
    model2_wins_black = match_dict["model2_wins_black"]
    draw = match_dict["draw"]
    total_games = match_dict["total_games"]
    game_num = match_dict["game_num"]

    print(
        f"\n{model1} win percentage : {(model1_wins_white + model1_wins_black) / total_games * 100}%"
    )
    print(
        f"{model2} win percentage : {(model2_wins_white + model2_wins_black) / total_games * 100}%"
    )
    print(f"Draw percentage : {draw / total_games * 100}%")
    print("=========================================")
    print(f"{model1} wins as white : {model1_wins_white}")
    print(f"{model1} wins as black : {model1_wins_black}")
    print()
    print(f"{model2} wins as white : {model2_wins_white}")
    print(f"{model2} wins as black : {model2_wins_black}")
    print("=========================================")
    print(f"{model1} wins total : {model1_wins_white + model1_wins_black}")
    print(f"{model2} wins total : {model2_wins_white + model2_wins_black}")
    print(f"Draws total : {draw}")
    print(f"Total games : {total_games}")
    print("=========================================")
    print(f"Played a total of {game_num * 2} games from the starting positions ({game_num} games each)")
    print("=========================================")

In [8]:
opening_moves = pd.read_csv("random_states.csv")

In [9]:
models = [
    stockfish("stockfish_800", 800, 6),
    stockfish("stockfish_2500", 2500, 12),

    vit("vit_800", 800, "../Model/vit_800_Elo_models/models/"),
    vit("vit_2500", 2500, "../Model/vit_2500_Elo_models/models/"),

    cnn("cnn_800", 800, "../Model/cnn_800_Elo_models/models/"),
    cnn("cnn_2500", 2500, "../Model/cnn_2500_Elo_models/models/"),

    random_agent("random")
]

In [16]:
pairings = []

for i in range(len(models)):
    for j in range(i + 1, len(models)):
        pairings.append((models[i], models[j]))

for i, matchup in enumerate(pairings, start=1):
    print(f"Match {i}: {matchup[0].name} vs {matchup[1].name}")

Match 1: stockfish_easy vs stockfish_medium
Match 2: stockfish_easy vs stockfish_hard
Match 3: stockfish_easy vs vit_800
Match 4: stockfish_easy vs vit_2500
Match 5: stockfish_easy vs cnn_800
Match 6: stockfish_easy vs cnn_2500
Match 7: stockfish_easy vs random
Match 8: stockfish_medium vs stockfish_hard
Match 9: stockfish_medium vs vit_800
Match 10: stockfish_medium vs vit_2500
Match 11: stockfish_medium vs cnn_800
Match 12: stockfish_medium vs cnn_2500
Match 13: stockfish_medium vs random
Match 14: stockfish_hard vs vit_800
Match 15: stockfish_hard vs vit_2500
Match 16: stockfish_hard vs cnn_800
Match 17: stockfish_hard vs cnn_2500
Match 18: stockfish_hard vs random
Match 19: vit_800 vs vit_2500
Match 20: vit_800 vs cnn_800
Match 21: vit_800 vs cnn_2500
Match 22: vit_800 vs random
Match 23: vit_2500 vs cnn_800
Match 24: vit_2500 vs cnn_2500
Match 25: vit_2500 vs random
Match 26: cnn_800 vs cnn_2500
Match 27: cnn_800 vs random
Match 28: cnn_2500 vs random


In [11]:
#Create a dataframe to store all the results
results = pd.DataFrame(columns = ["game_id", "model1_name", "model2_name", "model1_wins_white", "model1_wins_black", "model2_wins_white", "model2_wins_black", "draw", "total_games", "game_num"])

for run in range(10):
    #Shuffle the dataframe
    opening_moves = opening_moves.sample(frac=1).reset_index(drop=True)
    
    for id, comp in tqdm(enumerate(pairings)):
        game_details = play_games(comp[0], comp[1], 100, opening_moves)
        game_details["game_id"] = id
        results = results.append(game_details, ignore_index=True)

  0%|          | 0/100 [00:00<?, ?it/s]

  8%|▊         | 8/100 [00:02<00:25,  3.59it/s]


KeyboardInterrupt: 

In [ ]:
results.to_csv("new_results.csv", index=False)